# Checkpoint 5: Sector Analysis 2

- Create three separate data frames D1, D2 and D3 for each of the three countries containing the observations of funding type FT falling within the 5-15 million USD range. The three data frames should contain:
1. All the columns of the master_frame along with the primary sector and the main sector
2. The total number (or count) of investments for each main sector in a separate column
3. The total amount invested in each main sector in a separate column

In [1]:
# Loading libraries and reading data

import numpy as np
import pandas as pd

# Reading delimited file using pd.read_csv(filepath, sep, header)
# Using encoding = "ISO-8859-1"
companies = pd.read_csv("companies.txt", sep="\t", encoding = "ISO-8859-1")
rounds2 = pd.read_csv("rounds2.csv", encoding = "ISO-8859-1")
mapping_df = pd.read_csv("mapping.csv", encoding = "ISO-8859-1")

In [2]:
# rows having at least one missing value
companies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66368 entries, 0 to 66367
Data columns (total 10 columns):
permalink        66368 non-null object
name             66367 non-null object
homepage_url     61310 non-null object
category_list    63220 non-null object
status           66368 non-null object
country_code     59410 non-null object
state_code       57821 non-null object
region           58338 non-null object
city             58340 non-null object
founded_at       51147 non-null object
dtypes: object(10)
memory usage: 5.1+ MB


In [3]:
# remove Nan rows for category list
companies['category_list'].isnull().any(axis=0)
companies = companies[~pd.isnull(companies['category_list'])]
companies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 63220 entries, 0 to 66367
Data columns (total 10 columns):
permalink        63220 non-null object
name             63219 non-null object
homepage_url     59074 non-null object
category_list    63220 non-null object
status           63220 non-null object
country_code     57804 non-null object
state_code       56268 non-null object
region           56765 non-null object
city             56767 non-null object
founded_at       49711 non-null object
dtypes: object(10)
memory usage: 5.3+ MB


In [4]:
# cols having at least one missing value
companies['category_list'].isnull().any(axis=0)

False

In [5]:
# remove Nan rows for mapping_df.category_list
mapping_df['category_list'].isnull().any(axis=0)
mapping_df = mapping_df[~pd.isnull(mapping_df['category_list'])]
mapping_df.drop('Blanks', axis=1, inplace=True)
mapping_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 687 entries, 1 to 687
Data columns (total 9 columns):
category_list                              687 non-null object
Automotive & Sports                        687 non-null int64
Cleantech / Semiconductors                 687 non-null int64
Entertainment                              687 non-null int64
Health                                     687 non-null int64
Manufacturing                              687 non-null int64
News, Search and Messaging                 687 non-null int64
Others                                     687 non-null int64
Social, Finance, Analytics, Advertising    687 non-null int64
dtypes: int64(8), object(1)
memory usage: 53.7+ KB


In [6]:
mapping_df.nunique()

category_list                              687
Automotive & Sports                          2
Cleantech / Semiconductors                   2
Entertainment                                2
Health                                       2
Manufacturing                                2
News, Search and Messaging                   2
Others                                       2
Social, Finance, Analytics, Advertising      2
dtype: int64

In [7]:
# Merging the dataframes
# company_permalink,permalink is the common column/key, which has to be provided to the 'on' argument
# how = 'inner' makes sure that all the comapnies ids are reflectd in the master_frame

companies['permalink'] = companies['permalink'].astype(str).str.lower()
rounds2['company_permalink'] = rounds2['company_permalink'].astype(str).str.lower()

master_frame = pd.merge(companies, rounds2, how='inner', left_on='permalink', right_on='company_permalink')

In [8]:
# raised_amount_usd has considerable number of missing value.
# So, it will not help in finding average for funding_round_type
# Let's remove such rows from master_frame

# removing NaN Price rows
master_frame = master_frame[~np.isnan(master_frame['raised_amount_usd'])]

In [9]:
# Creating new column primary_sector by spliting '|' the category_list
master_frame['primary_sector'] = master_frame['category_list'].apply(lambda x : x.split('|')[0])

In [10]:
master_frame.head()

,permalink,name,homepage_url,category_list,status,country_code,state_code,region,city,founded_at,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd,primary_sector
0,/organization/-fame,#fame,http://livfame.com,Media,operating,IND,16,Mumbai,Mumbai,NaN,/organization/-fame,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,05-01-2015,10000000.0,Media
2,/organization/-qounter,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,operating,USA,DE,DE - Other,Delaware City,04-09-2014,/organization/-qounter,/funding-round/b44fbb94153f6cdef13083530bb48030,seed,NaN,01-03-2014,700000.0,Application Platforms
3,/organization/-the-one-of-them-inc-,"(THE) ONE of THEM,Inc.",http://oneofthem.jp,Apps|Games|Mobile,operating,NaN,NaN,NaN,NaN,NaN,/organization/-the-one-of-them-inc-,/funding-round/650b8f704416801069bb178a1418776b,venture,B,30-01-2014,3406878.0,Apps
4,/organization/0-6-com,0-6.com,http://www.0-6.com,Curated Web,operating,CHN,22,Beijing,Beijing,01-01-2007,/organization/0-6-com,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,19-03-2008,2000000.0,Curated Web
6,/organization/01games-technology,01Games Technology,http://www.01games.hk/,Games,operating,HKG,NaN,Hong Kong,Hong Kong,NaN,/organization/01games-technology,/funding-round/7d53696f2b4f607a2f2a8cbb83d01839,undisclosed,NaN,01-07-2014,41250.0,Games


In [11]:
# Step1: creating a series of main sector from mapping.csv
# Step2: Assigning the created series to mapping_df 
# Step3: Merge master_frame and mapping_df

mapping_df.set_index('category_list',inplace=True)
mapping_df_with_main_sector = mapping_df.idxmax(axis=1)

mapping_df = mapping_df.assign(main_sector=mapping_df_with_main_sector.values)
mapping_df = mapping_df.reset_index(drop=False)


master_primary_main_sector_df = pd.merge(master_frame, mapping_df, how='inner', on='category_list')


In [12]:
master_primary_main_sector_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41541 entries, 0 to 41540
Data columns (total 26 columns):
permalink                                  41541 non-null object
name                                       41540 non-null object
homepage_url                               39403 non-null object
category_list                              41541 non-null object
status                                     41541 non-null object
country_code                               39754 non-null object
state_code                                 39108 non-null object
region                                     39130 non-null object
city                                       39133 non-null object
founded_at                                 32530 non-null object
company_permalink                          41541 non-null object
funding_round_permalink                    41541 non-null object
funding_round_type                         41541 non-null object
funding_round_code                         127

In [13]:
# merged data frame
master_primary_main_sector_df.head()

,permalink,name,homepage_url,category_list,status,country_code,state_code,region,city,founded_at,...,primary_sector,Automotive & Sports,Cleantech / Semiconductors,Entertainment,Health,Manufacturing,"News, Search and Messaging",Others,"Social, Finance, Analytics, Advertising",main_sector
0,/organization/-fame,#fame,http://livfame.com,Media,operating,IND,16,Mumbai,Mumbai,NaN,...,Media,0,0,1,0,0,0,0,0,Entertainment
1,/organization/all-def-digital,All Def Digital,http://alldefdigital.com,Media,operating,USA,CA,Los Angeles,Los Angeles,NaN,...,Media,0,0,1,0,0,0,0,0,Entertainment
2,/organization/anthill-magazine,Anthill Magazine,http://anthillonline.com/,Media,operating,AUS,7,Melbourne,Melbourne,23-05-2009,...,Media,0,0,1,0,0,0,0,0,Entertainment
3,/organization/basel-switzerland,The Speedel Group,http://www.speedelgroup.com/,Media,closed,CHE,4,Basel,Basel,NaN,...,Media,0,0,1,0,0,0,0,0,Entertainment
4,/organization/basel-switzerland,The Speedel Group,http://www.speedelgroup.com/,Media,closed,CHE,4,Basel,Basel,NaN,...,Media,0,0,1,0,0,0,0,0,Entertainment


In [14]:
master_primary_main_sector_df.shape

(41541, 26)

In [15]:
# group by main_sector and count # of investment for each sector
master_primary_main_sector_df['total_no_of_investment'] = master_primary_main_sector_df.groupby(master_primary_main_sector_df['main_sector'])['funding_round_permalink'].transform('count')

# group by main_sector and count sum of amount invested **(in million USD)** for each sector
master_primary_main_sector_df['total_amount_invested_million_usd'] = master_primary_main_sector_df.groupby(master_primary_main_sector_df['main_sector'])['raised_amount_usd'].transform('sum')/1000000
    
master_primary_main_sector_df

,permalink,name,homepage_url,category_list,status,country_code,state_code,region,city,founded_at,...,Cleantech / Semiconductors,Entertainment,Health,Manufacturing,"News, Search and Messaging",Others,"Social, Finance, Analytics, Advertising",main_sector,total_no_of_investment,total_amount_invested_million_usd
0,/organization/-fame,#fame,http://livfame.com,Media,operating,IND,16,Mumbai,Mumbai,NaN,...,0,1,0,0,0,0,0,Entertainment,2488,20359.645846
1,/organization/all-def-digital,All Def Digital,http://alldefdigital.com,Media,operating,USA,CA,Los Angeles,Los Angeles,NaN,...,0,1,0,0,0,0,0,Entertainment,2488,20359.645846
2,/organization/anthill-magazine,Anthill Magazine,http://anthillonline.com/,Media,operating,AUS,7,Melbourne,Melbourne,23-05-2009,...,0,1,0,0,0,0,0,Entertainment,2488,20359.645846
3,/organization/basel-switzerland,The Speedel Group,http://www.speedelgroup.com/,Media,closed,CHE,4,Basel,Basel,NaN,...,0,1,0,0,0,0,0,Entertainment,2488,20359.645846
4,/organization/basel-switzerland,The Speedel Group,http://www.speedelgroup.com/,Media,closed,CHE,4,Basel,Basel,NaN,...,0,1,0,0,0,0,0,Entertainment,2488,20359.645846
5,/organization/digital-news-asia,Digital News Asia,http://www.digitalnewsasia.com/,Media,operating,MYS,12,Kuala Lumpur,Petaling Jaya,01-05-2012,...,0,1,0,0,0,0,0,Entertainment,2488,20359.645846
6,/organization/genius-monkey,Genius Monkey,http://geniusmonkey.com,Media,operating,USA,AZ,Phoenix,Tempe,NaN,...,0,1,0,0,0,0,0,Entertainment,2488,20359.645846
7,/organization/glass-media,Glass Media,http://www.glass-media.com,Media,closed,USA,TX,Dallas,Dallas,01-01-2012,...,0,1,0,0,0,0,0,Entertainment,2488,20359.645846
8,/organization/global-yodel,Global Yodel,http://www.globalyodel.com/,Media,operating,USA,WA,WA - Other,Anacortes,NaN,...,0,1,0,0,0,0,0,Entertainment,2488,20359.645846
9,/organization/hakka-labs,Hakka Labs,https://www.hakkalabs.co/,Media,operating,USA,NY,New York City,New York,01-01-2012,...,0,1,0,0,0,0,0,Entertainment,2488,20359.645846


In [16]:
# Filter to 'venture' funding_round_type 
master_primary_main_sector_df = master_primary_main_sector_df[(master_primary_main_sector_df['funding_round_type'] == 'venture') & (master_primary_main_sector_df['raised_amount_usd'].between(5000000, 15000000))]

# Create three separate data frames D1, D2 and D3

D1 = master_primary_main_sector_df[master_primary_main_sector_df['country_code'] == 'USA']
D2 = master_primary_main_sector_df[master_primary_main_sector_df['country_code'] == 'GBR']
D3 = master_primary_main_sector_df[master_primary_main_sector_df['country_code'] == 'IND']

print(D1.shape)

(5949, 28)


In [17]:
D1.head()

,permalink,name,homepage_url,category_list,status,country_code,state_code,region,city,founded_at,...,Cleantech / Semiconductors,Entertainment,Health,Manufacturing,"News, Search and Messaging",Others,"Social, Finance, Analytics, Advertising",main_sector,total_no_of_investment,total_amount_invested_million_usd
1,/organization/all-def-digital,All Def Digital,http://alldefdigital.com,Media,operating,USA,CA,Los Angeles,Los Angeles,NaN,...,0,1,0,0,0,0,0,Entertainment,2488,20359.645846
59,/organization/aeropost,Aeropost,http://aeropost.com,Curated Web,operating,USA,FL,Miami,Miami,01-01-1986,...,0,0,0,0,1,0,0,"News, Search and Messaging",4331,30338.258605
63,/organization/akonix-systems,Akonix Systems,http://www.akonix.com,Curated Web,operating,USA,CA,San Diego,San Diego,01-01-2000,...,0,0,0,0,1,0,0,"News, Search and Messaging",4331,30338.258605
67,/organization/amberpoint,AmberPoint,http://www.amberpoint.com,Curated Web,acquired,USA,CA,SF Bay Area,Oakland,NaN,...,0,0,0,0,1,0,0,"News, Search and Messaging",4331,30338.258605
68,/organization/amberpoint,AmberPoint,http://www.amberpoint.com,Curated Web,acquired,USA,CA,SF Bay Area,Oakland,NaN,...,0,0,0,0,1,0,0,"News, Search and Messaging",4331,30338.258605


In [18]:
# Total number of investments 
print(D1['funding_round_permalink'].shape)
print(D2['funding_round_permalink'].shape)
print(D3['funding_round_permalink'].shape)

(5949,)
(337,)
(173,)


In [19]:
# Total amount of investment (USD)

print(D1['raised_amount_usd'].sum())
print(D2['raised_amount_usd'].sum())
print(D3['raised_amount_usd'].sum())


53272315512.0
2971451457.0
1517298277.0


In [20]:
# Top Sector name (no. of investment-wise)
# Second Sector name (no. of investment-wise)
# Third Sector name (no. of investment-wise)
# Number of investments in top sector (3)
# Number of investments in second sector (4)
# Number of investments in third sector (5)

print(D1.groupby('main_sector')['total_no_of_investment'].count().sort_values(ascending=False))
print(D2.groupby('main_sector')['total_no_of_investment'].count().sort_values(ascending=False))
print(D3.groupby('main_sector')['total_no_of_investment'].count().sort_values(ascending=False))


main_sector
Others                                     2069
Cleantech / Semiconductors                 1737
Health                                      601
News, Search and Messaging                  552
Manufacturing                               411
Social, Finance, Analytics, Advertising     343
Entertainment                               196
Automotive & Sports                          40
Name: total_no_of_investment, dtype: int64
main_sector
Cleantech / Semiconductors                 107
Others                                     106
News, Search and Messaging                  31
Manufacturing                               29
Entertainment                               28
Social, Finance, Analytics, Advertising     20
Health                                      13
Automotive & Sports                          3
Name: total_no_of_investment, dtype: int64
main_sector
Others                                     74
News, Search and Messaging                 31
Entertainment             

In [29]:
# For point 3 (top sector count-wise), which company received the highest investment?
#D1 = D1[D1['main_sector'] == 'Others'] -- Can be used for filtering the data even further

print(D1.groupby(['name','main_sector'])['total_no_of_investment'].count().sort_values(ascending=False))
print(D2.groupby(['name','main_sector'])['total_no_of_investment'].count().sort_values(ascending=False))
print(D3.groupby(['name','main_sector'])['total_no_of_investment'].count().sort_values(ascending=False))

name                       main_sector                            
Tigo Energy                Cleantech / Semiconductors                 8
ForSight Labs              Cleantech / Semiconductors                 7
EndoGastric Solutions      Health                                     7
YuMe                       Social, Finance, Analytics, Advertising    6
Virtustream                Others                                     6
ChoiceStream               Social, Finance, Analytics, Advertising    6
SCIenergy                  Cleantech / Semiconductors                 6
Capella Photonics          Others                                     6
Bit9                       Others                                     5
Ocular Therapeutix         Cleantech / Semiconductors                 5
Visible Measures           Social, Finance, Analytics, Advertising    5
Carbon Design Systems      Others                                     5
Sebacia                    News, Search and Messaging                